# Competitive coevolution with GEP

In [ ]:
experiment_name = 'only_gep'

In [ ]:
import random
random.seed(456)

## Regex for test

In [ ]:
input_regex = '((a|bb?)|(a|ba?)|([0-9]|.ab))'

params = {
    'range': ['0-9'],
    'repeat': ['0,1'],
}
    
nodes = {
    # functions (except any)
    -1: 'params', 
    0: 'seq', 
    1: 'atom', 
    2: 'any', 
    3: 'repeat', 
    4: 'alt', 
    5: 'altgroup', 
    6: 'group', 
    7: 'range', 
    8: 'escape', 

    # terminals (get by input regex)
    9: 'a', 
    10: 'b',
}

In [ ]:
from coevolutionary.utils import Utils
from coevolutionary.utils import regex_process

test_strings = Utils.get_test_strings(
    input_regex=input_regex,
    n_fuzzy_strings=5
)

X, Y = Utils.create_training_set(
    test_strings=test_strings,
    original_regex=input_regex,
    process_func=regex_process,
)  

## Run

In [ ]:
from coevolutionary.manager import CompetitiveManager
from coevolutionary.algorithms.gep import GEPAlgorithm
from coevolutionary.utils.terminals import get_terminals

In [ ]:
cm = CompetitiveManager(
    adaptive_interval=5,
    shared_resource=5_000,
    verbose=False,
    problem='min',
    survive_schema='best',
    social_card=0.3,
    penalty=0.10,
    experiment_name=experiment_name,
)

In [ ]:
params_cases = [
    {'population_length': 100, 'genes_n': 4, 'head_n': 2, 'n_elites': 3},
    {'population_length': 100, 'genes_n': 3, 'head_n': 3, 'n_elites': 10},
    {'population_length': 100, 'genes_n': 4, 'head_n': 2, 'n_elites': 10},
    {'population_length': 100, 'genes_n': 5, 'head_n': 3, 'n_elites': 3},
]

### Run algorithms in coevolution

In [ ]:
terminals = get_terminals(
    add_digits=False,
    add_lower_latin_letters=False,
    add_custom_symbols=True,
    custom_symbols=['a', 'b', 'any', 'range', 'escape']
)

for i, case in enumerate(params_cases):
    gep_object = GEPAlgorithm(
        X=X,
        Y=Y,
        n_iter=100,
        terminals=terminals,
        params=params,
    )
    
    cm.add_algorithm(
        name=f'gep_{i}_coev',
        init_params=case,
        init_algorithm=gep_object.init_algorithm,
        init_population=gep_object.init_population,
        get_fitness_population=gep_object.get_fitness_population,
        select_population=gep_object.select_population,
        recombine_population=gep_object.recombine_population,
        mutate_population=gep_object.mutate_population,
    )

In [ ]:
cm.run_coevolution()

In [ ]:
coevolution_algorithm_history = cm.algorithm_history
population_qualities_history = cm.population_qualities_history
coev_names = cm.get_algorithm_names()
best_alg_statistics = cm.get_winner_statistics()

In [ ]:
from coevolutionary.utils.visualization import visualize

visualize(
    algorithm_history=coevolution_algorithm_history,
    algorithm_names=coev_names
)

In [ ]:
import matplotlib.pyplot as plt
from coevolutionary.utils.visualization import visualize_algorithms_quality

plt.rcParams["figure.figsize"] = (10, 3)
visualize_algorithms_quality(
    population_qualities_history=population_qualities_history,
    algorithm_names=coev_names
)

In [ ]:
print('Best individual:', cm.get_best_individual())

### Run algorithms separately

In [ ]:
separately_algorithm_history = {}
sep_names = []

In [ ]:
from coevolutionary.utils.tests import check_wilcoxon

for i, case in enumerate(params_cases):
    cm = CompetitiveManager(
        adaptive_interval=2,
        shared_resource=5_000,
        verbose=False,
        problem='min',
        survive_schema='best',
        social_card=0.3,
        penalty=0.10,
        experiment_name=experiment_name,
    )
    
    gep_object = GEPAlgorithm(
        X=X,
        Y=Y,
        n_iter=100,
        terminals=terminals,
        params=params,
    )
    
    cm.add_algorithm(
        name=f'gep_{i}_sep',
        init_params=case,
        init_algorithm=gep_object.init_algorithm,
        init_population=gep_object.init_population,
        get_fitness_population=gep_object.get_fitness_population,
        select_population=gep_object.select_population,
        recombine_population=gep_object.recombine_population,
        mutate_population=gep_object.mutate_population,
    )
    
    # overload run
    cm.run_coevolution()
    
    separately_algorithm_history[i] = cm.algorithm_history[0]
    sep_names.append(f'gep_{i}_sep')
    
    print('\nBest individual: ', cm.get_best_individual())
    
    check_wilcoxon(
        history_a=cm.algorithm_history,
        history_b=best_alg_statistics,
        a_name=f'gep_{i}_sep',
        b_name=f'best of coevolutionary gep',
        a_index=0,
        b_index=i,
        metric_number=1,
        metric_name='minimum'
    )

In [ ]:
from coevolutionary.utils.visualization import visualize

plt.rcParams["figure.figsize"] = (15, 10)
visualize(
    algorithm_history=separately_algorithm_history,
    algorithm_names=sep_names
)